In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import GroupShuffleSplit

In [2]:
hct_dirs = ["/mnt/dataHDD/chris/m6anet_data_replicates/GIS_Hct116_directRNA_Rep2-Run1/"]

hek293_dirs = ["/mnt/dataHDD/chris/m6anet_data_replicates/GohGIS_Hek293T_directRNA_Rep2/"]

hct_replicates = [os.path.join(hct_dir, "data.readcount.labelled") for hct_dir in hct_dirs]

hek293_replicates = [os.path.join(hek293_dir, "data.readcount.labelled") for hek293_dir in hek293_dirs]


In [3]:
hct_df_filtered = None
join_keys = ["chr", "gene_id", "genomic_position", "transcript_id", "transcript_position", "modification_status"]

for rep in hct_replicates:
    rep_df = pd.read_csv(rep)
    rep_df = rep_df[rep_df["n_reads"] >= 20].reset_index(drop=True)
    rep_df = rep_df.rename(columns={'n_reads': "n_reads_{}".format(rep.split("/")[-2])})\
        .set_index(join_keys)
    
    if hct_df_filtered is None:
        hct_df_filtered = rep_df
    else:
        hct_df_filtered = hct_df_filtered.merge(rep_df, on=join_keys)

hct_df_filtered = hct_df_filtered.reset_index()

        
hek293_df_filtered = None
for rep in hek293_replicates:
    rep_df = pd.read_csv(rep)
    rep_df = rep_df[rep_df["n_reads"] >= 20].reset_index(drop=True)
    rep_df = rep_df.rename(columns={'n_reads': "n_reads_{}".format(rep.split("/")[-2])})\
        .set_index(["gene_id", "genomic_position", "transcript_id", "transcript_position"])
    if hek293_df_filtered is None:
        hek293_df_filtered = rep_df
    else:
        hek293_df_filtered = hek293_df_filtered.merge(rep_df, on=join_keys)
        
hek293_df_filtered = hek293_df_filtered.reset_index()        

print("Total HCT116 positions after filtering: {}".format(hct_df_filtered.shape[0]))
print("Total HEK293T positions after filtering: {}".format(hek293_df_filtered.shape[0]))

hct_genes = hct_df_filtered["gene_id"].unique()
hek293_genes = hek293_df_filtered["gene_id"].unique()
shared_genes = np.intersect1d(hek293_genes, hct_genes)
print("There are {} unique genes in HCT cell lines".format(hct_genes.shape[0]))
print("There are {} unique genes in HEK293T cell lines".format(hek293_genes.shape[0]))
print("Shared genes: {}".format(shared_genes.shape[0]))

Total HCT116 positions after filtering: 121838
Total HEK293T positions after filtering: 90810
There are 3852 unique genes in HCT cell lines
There are 3149 unique genes in HEK293T cell lines
Shared genes: 2515


We shall randomly sample 500 of these shared genes to make a test set and allocate the rest as training set for each cell line

In [4]:
np.random.seed(0)
test_genes = np.random.choice(shared_genes, 500, replace=False)
print("Train information: ")
hct116_train_mod_counts = np.unique(hct_df_filtered[~hct_df_filtered["gene_id"].isin(test_genes)]["modification_status"], 
                                    return_counts=True)[1]
hek293t_train_mod_counts = np.unique(hek293_df_filtered[~hek293_df_filtered["gene_id"].isin(test_genes)]["modification_status"],
                                     return_counts=True)[1]
print("HCT116: {} unmodified and {} modified positions".format(hct116_train_mod_counts[0], hct116_train_mod_counts[1]))
print("HEK293T: {} unmodified and {} modified positions".format(hek293t_train_mod_counts[0], hek293t_train_mod_counts[1]))
print("==============================")
print("Test information: ")
hct116_test_mod_counts = np.unique(hct_df_filtered[hct_df_filtered["gene_id"].isin(test_genes)]["modification_status"], 
                                    return_counts=True)[1]
hek293t_test_mod_counts = np.unique(hek293_df_filtered[hek293_df_filtered["gene_id"].isin(test_genes)]["modification_status"],
                                     return_counts=True)[1]
print("HCT116: {} unmodified and {} modified positions".format(hct116_test_mod_counts[0], hct116_test_mod_counts[1]))
print("HEK293T: {} unmodified and {} modified positions".format(hek293t_test_mod_counts[0], hek293t_test_mod_counts[1]))
print("==============================")

hct_train_val = hct_df_filtered[~hct_df_filtered["gene_id"].isin(test_genes)]
train_val_sites = hct_df_filtered[~hct_df_filtered["gene_id"].isin(test_genes)].index
train_index, val_index = next(GroupShuffleSplit(n_splits=5).split(train_val_sites, hct_train_val["modification_status"],
                                                                  groups=hct_train_val["gene_id"]))

train_sites, val_sites = train_val_sites[train_index], train_val_sites[val_index]
test_sites = hct_df_filtered[hct_df_filtered["gene_id"].isin(test_genes)].index

hct_df_filtered["set_type"] = "NA"
hct_df_filtered.loc[train_sites, "set_type"] = np.repeat("Train", len(train_sites))
hct_df_filtered.loc[val_sites, "set_type"] = np.repeat("Val", len(val_sites))
hct_df_filtered.loc[test_sites, "set_type"] = np.repeat("Test", len(test_sites))

_, modified_counts_train = np.unique(hct_df_filtered.iloc[train_sites]["modification_status"], return_counts=True)
_, modified_counts_val = np.unique(hct_df_filtered.iloc[val_sites]["modification_status"], return_counts=True)
_, modified_counts_test = np.unique(hct_df_filtered.iloc[test_sites]["modification_status"], return_counts=True)

print("======================================================")

print("Train test split for HCT116 cell lines data:")
print("There are {} train sites with {} unmodified sites, {} modified sites".format(len(train_sites), 
                                                                                    modified_counts_train[0], 
                                                                                    modified_counts_train[1]))
print("There are {} val sites with {} unmodified sites, {} modified sites".format(len(val_sites), 
                                                                                    modified_counts_val[0], 
                                                                                    modified_counts_val[1]))
print("There are {} test sites with {} unmodified sites, {} modified sites".format(len(test_sites), 
                                                                                    modified_counts_test[0], 
                                                                                    modified_counts_test[1]))
print("======================================================")

hek293_train_val = hek293_df_filtered[~hek293_df_filtered["gene_id"].isin(test_genes)]
train_val_sites = hek293_df_filtered[~hek293_df_filtered["gene_id"].isin(test_genes)].index
train_index, val_index = next(GroupShuffleSplit(n_splits=5).split(train_val_sites, 
                                                                  hek293_train_val["modification_status"],
                                                                  groups=hek293_train_val["gene_id"]))

train_sites, val_sites = train_val_sites[train_index], train_val_sites[val_index] 
test_sites = hek293_df_filtered[hek293_df_filtered["gene_id"].isin(test_genes)].index

hek293_df_filtered["set_type"] = "NA"
hek293_df_filtered.loc[train_sites, "set_type"] = np.repeat("Train", len(train_sites))
hek293_df_filtered.loc[val_sites, "set_type"] = np.repeat("Val", len(val_sites))
hek293_df_filtered.loc[test_sites, "set_type"] = np.repeat("Test", len(test_sites))

_, modified_counts_train = np.unique(hek293_df_filtered.iloc[train_sites]["modification_status"], return_counts=True)
_, modified_counts_val = np.unique(hek293_df_filtered.iloc[val_sites]["modification_status"], return_counts=True)
_, modified_counts_test = np.unique(hek293_df_filtered.iloc[test_sites]["modification_status"], return_counts=True)

print("======================================================")

print("Train test split for HEK293T cell lines data:")
print("There are {} train sites with {} unmodified sites, {} modified sites".format(len(train_sites), 
                                                                                    modified_counts_train[0], 
                                                                                    modified_counts_train[1]))
print("There are {} val sites with {} unmodified sites, {} modified sites".format(len(val_sites), 
                                                                                    modified_counts_val[0], 
                                                                                    modified_counts_val[1]))
print("There are {} test sites with {} unmodified sites, {} modified sites".format(len(test_sites), 
                                                                                    modified_counts_test[0], 
                                                                                    modified_counts_test[1]))
print("======================================================")


Train information: 
HCT116: 99542 unmodified and 4753 modified positions
HEK293T: 69399 unmodified and 5621 modified positions
Test information: 
HCT116: 16821 unmodified and 722 modified positions
HEK293T: 14818 unmodified and 972 modified positions
Train test split for HCT116 cell lines data:
There are 83242 train sites with 79445 unmodified sites, 3797 modified sites
There are 21053 val sites with 20097 unmodified sites, 956 modified sites
There are 17543 test sites with 16821 unmodified sites, 722 modified sites
Train test split for HEK293T cell lines data:
There are 60322 train sites with 55853 unmodified sites, 4469 modified sites
There are 14698 val sites with 13546 unmodified sites, 1152 modified sites
There are 15790 test sites with 14818 unmodified sites, 972 modified sites


In [5]:
from shutil import copyfile


In [6]:
save_dir = "/mnt/dataHDD/chris/m6anet_data_replicates_simple"

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for replicate in np.concatenate([hct_dirs, hek293_dirs]):
    rep_dir = os.path.join(save_dir, replicate.split("/")[-2])    
    if not os.path.exists(rep_dir):
        os.mkdir(rep_dir)
        
#     copyfile(os.path.join(replicate, "data.json"), os.path.join(rep_dir, "data.json"))


In [7]:
hct_df_filtered_rep = hct_df_filtered.set_index(join_keys).copy()
for rep in hct_replicates:
    rep_df = pd.read_csv(rep)
    rep_df = rep_df[rep_df["n_reads"] >= 20].reset_index(drop=True).set_index(join_keys)[["n_reads", "kmer"]]
    rep_df = rep_df.merge(hct_df_filtered_rep["set_type"], on=join_keys).reset_index()
    
    save_path = os.path.join(save_dir, rep.split("/")[-2], "data.readcount.labelled")
    rep_df.to_csv(save_path,index=False)
    print(np.unique(rep_df["set_type"], return_counts=True))
    print("======")


(array(['Test', 'Train', 'Val'], dtype=object), array([17543, 83242, 21053]))


In [8]:
join_keys = ["chr", "gene_id", "genomic_position", "transcript_id", "transcript_position", "modification_status"]
hek293_df_filtered_rep = hek293_df_filtered.set_index(join_keys).copy()
for rep in hek293_replicates:
    rep_df = pd.read_csv(rep)
    rep_df = rep_df[rep_df["n_reads"] >= 20].reset_index(drop=True).set_index(join_keys)[["n_reads", "kmer"]]
    rep_df = rep_df.merge(hek293_df_filtered_rep["set_type"], on=join_keys).reset_index()
    save_path = os.path.join(save_dir, rep.split("/")[-2], "data.readcount.labelled")
    print(save_path)
    rep_df.to_csv(save_path,index=False)
    print(np.unique(rep_df["set_type"], return_counts=True))
    print("======")


/mnt/dataHDD/chris/m6anet_data_replicates_simple/GohGIS_Hek293T_directRNA_Rep2/data.readcount.labelled
(array(['Test', 'Train', 'Val'], dtype=object), array([15790, 60322, 14698]))
